This code works through the GEDI subsetter version 0.6.0. This converts a geojson of tiles over a study area to temp geojson files to feed into the subsetter

In [1]:
#Open the MAAP host
from maap.maap import MAAP
maap = MAAP(maap_host='api.maap-project.org')


In [2]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import geopandas as gpd
import shapely
import glob
import os
import pandas as pd
import fiona

In [3]:
import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [4]:
#Needed For L2B
#Create empty variables to fill with all 29 appearances of cover_z* and pai_z* 
variables = []
for n in np.arange(0, 30,1):
    variables.append('cover_z' + str(n))
    variables.append('pai_z' + str(n))

In [6]:
cwd = os.getcwd()
cwd

'/projects/GEDI_PA/MAAP_GEDIAnalysis'

In [7]:
os.chdir('/projects/GEDI_PA')

In [31]:
result_folder = "/projects/shared-buckets/abarenblitt/SEN_Tiles"
AOIS = [file for file in os.listdir(result_folder) if file.endswith('.geojson')]
print(AOIS)
# for each_aoi in AOIS:
#     aoi_url = os.path.join('s3://maap-ops-workspace/shared/abarenblitt/SEN_Tiles', each_aoi)
#     print(aoi_url)

['N12.30728831742391W-10.52951992770953.geojson', 'N12.30728831742391W-11.52951992770953.geojson', 'N12.30728831742391W-12.52951992770953.geojson', 'N12.30728831742391W-13.52951992770953.geojson', 'N12.30728831742391W-14.52951992770953.geojson', 'N12.30728831742391W-15.52951992770953.geojson', 'N12.30728831742391W-16.52951992770953.geojson', 'N12.30728831742391W-17.52951992770953.geojson', 'N13.30728831742391W-10.52951992770953.geojson', 'N13.30728831742391W-11.52951992770953.geojson', 'N13.30728831742391W-12.52951992770953.geojson', 'N13.30728831742391W-13.52951992770953.geojson', 'N13.30728831742391W-14.52951992770953.geojson', 'N13.30728831742391W-15.52951992770953.geojson', 'N13.30728831742391W-16.52951992770953.geojson', 'N13.30728831742391W-17.52951992770953.geojson', 'N14.30728831742391W-10.52951992770953.geojson', 'N14.30728831742391W-11.52951992770953.geojson', 'N14.30728831742391W-12.52951992770953.geojson', 'N14.30728831742391W-13.52951992770953.geojson', 'N14.30728831742391

In [29]:
PRODUCTS = ["L4A","L2A","L2B"]

In [34]:
#Set up run to pull products for all GEDI products simultaneously
#To run for only 1 or 2 products, change "PRODUCTS"

for aoi in AOIS:
    if "L4A" in PRODUCTS: 
        print("Thanks, I'll run the GEDI L4A subsetter!!")
        aoi_url = os.path.join('s3://maap-ops-workspace/shared/abarenblitt/SEN_tiles', aoi)
        # aoi_url = os.path.basename(each_aoi)
        inputs = dict(
           aoi=aoi_url,
           doi="L4A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lat_lowestmode,lon_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,agbd_pi_upper,agbd_pi_lower,sensitivity,predictor_limit_flag,response_limit_flag,selected_algorithm,selected_mode,selected_mode_flag",
           query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95 and `geolocation/sensitivity_a2` > 0.95",
           limit = 10_000,
           output= os.path.basename(aoi).split('.')[0] + "_L4A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="abarenblitt",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

    if "L2B" in PRODUCTS: 
        print("Thanks, I'll run the GEDI L2B subsetter!!")
        aoi_url = os.path.join('s3://maap-ops-workspace/shared/abarenblitt/SEN_tiles', aoi)
        inputs = dict(
           aoi=aoi_url,
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100, l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover, pai,fhd_normal,"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 10_000,
           output= os.path.basename(aoi).split('.')[0] + "_L2B.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="abarenblitt",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result


    if "L2A" in PRODUCTS: 
        print("Thanks, I'll run the GEDI L2A subsetter!!")
        aoi_url = os.path.join('s3://maap-ops-workspace/shared/abarenblitt/SEN_tiles', aoi)
        inputs = dict(
           aoi=aoi_url,
           doi="L2A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="geolocation/sensitivity_a2,shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98",
           query="quality_flag == 1 and geolocation.sensitivity_a2 > 0.95", ## geolocation.sensitivity_a2 > 0.95
           limit = 10_000,
           output= os.path.basename(aoi).split('.')[0] + "_L2A.csv"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.7.0",
            queue="maap-dps-worker-32gb",
            username="abarenblitt",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result
        
# Need to choose 1 of 6 algorithms for ground-finding for each shot CHECK WITH LAURA

Thanks, I'll run the GEDI L4A subsetter!!
Thanks, I'll run the GEDI L2B subsetter!!
Thanks, I'll run the GEDI L2A subsetter!!
Thanks, I'll run the GEDI L4A subsetter!!
Thanks, I'll run the GEDI L2B subsetter!!
Thanks, I'll run the GEDI L2A subsetter!!
Thanks, I'll run the GEDI L4A subsetter!!
Thanks, I'll run the GEDI L2B subsetter!!
Thanks, I'll run the GEDI L2A subsetter!!
Thanks, I'll run the GEDI L4A subsetter!!
Thanks, I'll run the GEDI L2B subsetter!!
Thanks, I'll run the GEDI L2A subsetter!!
Thanks, I'll run the GEDI L4A subsetter!!
Thanks, I'll run the GEDI L2B subsetter!!
Thanks, I'll run the GEDI L2A subsetter!!
Thanks, I'll run the GEDI L4A subsetter!!
Thanks, I'll run the GEDI L2B subsetter!!
Thanks, I'll run the GEDI L2A subsetter!!
Thanks, I'll run the GEDI L4A subsetter!!
Thanks, I'll run the GEDI L2B subsetter!!
Thanks, I'll run the GEDI L2A subsetter!!
Thanks, I'll run the GEDI L4A subsetter!!
Thanks, I'll run the GEDI L2B subsetter!!
Thanks, I'll run the GEDI L2A subs

In [8]:
job_id or result

'9df0a35f-29ee-43d8-807b-9d177da4f44d'

In [9]:
import os
import subprocess

root_dir = '/projects/my-private-bucket/dps_output/gedi-subset/0.6.2/gedi-subset/2024/02/27' #Can now set name
out_dir = '~/GEDI_PA/Matching_Layers/SEN'
for dirName, subdirList, fileList in os.walk(root_dir):
    print('Found directory: %s' % dirName)
    for fname in fileList:
        if fname.endswith('.gpkg'):
            print(fname)
            subprocess.call('cp ' + os.path.join(dirName, fname) + ' ' + os.path.join(out_dir, fname),shell=True)

Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.2/gedi-subset/2024/02/27
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.2/gedi-subset/2024/02/27/12
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.2/gedi-subset/2024/02/27/12/46
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.2/gedi-subset/2024/02/27/12/46/09
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.2/gedi-subset/2024/02/27/12/46/09/010752
SEN_admin_L4A.gpkg
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.2/gedi-subset/2024/02/27/16
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.2/gedi-subset/2024/02/27/16/18
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.2/gedi-subset/2024/02/27/16/18/27
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.2/gedi-subset/2024/02/27/16/18/27/006542
SEN_admin_L2A.gpkg


In [21]:
# LIST_OF_OUTPUTS = glob.glob("/projects/my-private-bucket/dps_output/gedi-subset/0.6.2/GGNP/*.gpkg")

LIST_OF_COUNTRIES = ['GGNP/GGNPBorder']


for each_country in LIST_OF_COUNTRIES:
    L2A = gpd.read_file('/projects/my-private-bucket/dps_output/gedi-subset/0.6.2/' + each_country +'_L2A.gpkg')
    L2B = gpd.read_file('/projects/my-private-bucket/dps_output/gedi-subset/0.6.2/'+ each_country +'_L2B.gpkg')
    L4A = gpd.read_file('/projects/my-private-bucket/dps_output/gedi-subset/0.6.2/'+ each_country +'_L4A.gpkg')
    combo = L2A.merge(L2B,on='shot_number').merge(L4A,on='shot_number')
    combo

type(combo)

pandas.core.frame.DataFrame

In [31]:
list(combo.columns)

['filename_x',
 'rh25',
 'rh98',
 'lat_lowestmode_x',
 'shot_number',
 'rh50',
 'rh90',
 'rh75',
 'lon_lowestmode_x',
 'geolocation/sensitivity_a2',
 'geometry_x',
 'filename_y',
 'pai_z9',
 'pai_z10',
 'cover_z20',
 'pai_z29',
 'cover_z10',
 'pai_z18',
 'cover_z1',
 'pai_z24',
 'cover_z19',
 'pai',
 'cover_z4',
 'pai_z14',
 'cover_z25',
 'cover_z29',
 'cover_z5',
 'pai_z17',
 'fhd_normal',
 'cover_z18',
 'rh100',
 'pai_z26',
 'cover_z17',
 'pai_z6',
 'cover_z13',
 'pai_z27',
 'pai_z22',
 'pai_z5',
 'pai_z20',
 'geolocation/lat_lowestmode',
 'pai_z8',
 'l2b_quality_flag',
 'cover_z26',
 'cover_z6',
 'land_cover_data/landsat_treecover',
 'cover_z11',
 'pai_z12',
 'pai_z1',
 'cover_z14',
 'cover_z9',
 'pai_z11',
 'pai_z23',
 'cover_z7',
 'pai_z0',
 'cover_z12',
 'pai_z4',
 'geolocation/lon_lowestmode',
 'cover_z0',
 'pai_z16',
 'pai_z15',
 'cover_z16',
 'pai_z13',
 'cover_z2',
 'sensitivity_x',
 'pai_z7',
 'pai_z28',
 'pai_z2',
 'cover_z8',
 'pai_z19',
 'cover_z21',
 'cover_z27',
 'cover